# Q1. Install MLflow

In [1]:
# Q1
! mlflow --version

mlflow, version 2.13.0


# Q2. Download and preprocess the data

In [2]:
links = [
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet"
]

# download data to data subdirectory replace if exists
for link in links:
    ! wget -N -P data $link

--2024-05-24 14:31:04--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.85.39.97, 52.85.39.117, 52.85.39.65, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.85.39.97|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘data/green_tripdata_2023-01.parquet’ not modified on server. Omitting download.

--2024-05-24 14:31:05--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.85.39.117, 52.85.39.97, 52.85.39.65, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.85.39.117|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘data/green_tripdata_2023-02.parquet’ not modified on server. Omitting download.

--2024-05-24 14:31:06--  https://d37ci6vzurychx.clo

In [3]:
# download preprocess_data.py
! wget -N https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2024/02-experiment-tracking/homework/preprocess_data.py

--2024-05-24 14:31:13--  https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2024/02-experiment-tracking/homework/preprocess_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2510 (2.5K) [text/plain]
Saving to: ‘preprocess_data.py’

preprocess_data.py  100%[===================>]   2.45K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2024-05-24 14:31:13 (41.0 MB/s) - ‘preprocess_data.py’ saved [2510/2510]



In [5]:
! python preprocess_data.py --raw_data_path data --dest_path ./output

In [7]:
# Q2
# number of files in output directory
import os
output_dir = './output'
num_files = len([name for name in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, name))])
print(f"Number of files in output directory: {num_files}")

Number of files in output directory: 4


# Q3. Train a model with autolog

In [8]:
# download train.py
# ! wget -N https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2024/02-experiment-tracking/homework/train.py

In [9]:
# run modified train.py
! python train.py

2024/05/24 14:32:31 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/05/24 14:32:32 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/24 14:32:46 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/envs/hw2-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."


In [10]:
# Q3
# get min_samples_split from last run
import mlflow
import pandas as pd

# Get last run
experiments = mlflow.search_runs()
last_run = experiments[experiments['status'] == 'FINISHED'].sort_values(by='end_time', ascending=False).head(1)

# Get run_id
run_id = last_run['run_id'].values[0]

# Get run
run = mlflow.get_run(run_id)

# Get run data
data = run.data

# Get params
params = data.params

# Get min_samples_split
min_samples_split = params['min_samples_split']
print(f"min_samples_split: {min_samples_split}")

min_samples_split: 2


# Q4. Launch the tracking server locally

Launch mlflow locally select a `hw2.sqlite` db for the backend store and
a folder called `artifacts` for the artifacts store
```bash
mlflow ui --backend-store-uri sqlite:///hw2.sqlite --artifacts-destination ./artifacts
```
References:
https://stackoverflow.com/questions/75057477/mlflow-server-difference-between-default-artifact-root-and-artifacts
https://mlflow.org/docs/latest/cli.html#mlflow-artifacts

In [11]:
# ! mlflow ui --backend-store-uri sqlite:///hw2.sqlite --artifacts-destination ./artifacts

In [12]:
# Q4 answer: artifacts-destination

# Q5. Tune model hyperparameters

In [13]:
# download hpo.py
# ! wget -N https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2024/02-experiment-tracking/homework/hpo.py

In [20]:
# Q5
! python hpo.py

100%|██████████| 15/15 [01:00<00:00,  4.02s/trial, best loss: 5.335419588556921]


# Q6. Promote the best model to the model registry

In [21]:
# download register_model.py
# ! wget -N https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2024/02-experiment-tracking/homework/register_model.py

In [25]:
# Q6
! python register_model.py

2024/05/24 14:44:30 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-model' does not exist. Creating a new experiment.
urllib3.exceptions.ResponseError: too many 503 error responses

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/envs/hw2-env/lib/python3.9/site-packages/requests/adapters.py", line 589, in send
    resp = conn.urlopen(
  File "/opt/conda/envs/hw2-env/lib/python3.9/site-packages/urllib3/connectionpool.py", line 948, in urlopen
    return self.urlopen(
  File "/opt/conda/envs/hw2-env/lib/python3.9/site-packages/urllib3/connectionpool.py", line 948, in urlopen
    return self.urlopen(
  File "/opt/conda/envs/hw2-env/lib/python3.9/site-packages/urllib3/connectionpool.py", line 948, in urlopen
    return self.urlopen(
  [Previous line repeated 2 more times]
  File "/opt/conda/envs/hw2-env/lib/python3.9/site-packages/urllib3/connectionpool.py", line 938, in urlopen
    retries = 